In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50

In [2]:
dataset_path = "dataset"

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='training')

val_generator = datagen.flow_from_directory(
    dataset_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='validation')

Found 1003 images belonging to 4 classes.
Found 250 images belonging to 4 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes: Real Tumor, Real Non-Tumor, Deepfake Tumor, Deepfake Non-Tumor
])

D:\FreeLancing_Projects\tumordetect-ai\env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
history = model.fit(train_generator, validation_data=val_generator, epochs=15)

D:\FreeLancing_Projects\tumordetect-ai\env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.7769 - loss: 0.8206 - val_accuracy: 0.9520 - val_loss: 0.1859
Epoch 2/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 944ms/step - accuracy: 0.9617 - loss: 0.1114 - val_accuracy: 0.9560 - val_loss: 0.1535
Epoch 3/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9724 - loss: 0.0876 - val_accuracy: 0.9440 - val_loss: 0.1600
Epoch 4/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9580 - loss: 0.0906 - val_accuracy: 0.9560 - val_loss: 0.1744
Epoch 5/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 32s 983ms/step - accuracy: 0.9768 - loss: 0.0784 - val_accuracy: 0.9560 - val_loss: 0.1769
Epoch 6/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 943ms/step - accuracy: 0.9943 - loss: 0.0753 - val_accuracy: 0.9680 - val_loss: 0.1695
Epoch 7/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.9889 - loss: 0.0306 - val_accuracy: 0.9560 - val_loss: 0.2385
Epoch 8/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 762s 25s/step - accuracy: 0.9973 - loss: 0.0148 - val_accuracy: 0.9520 

In [7]:
model.save("tumor_classification_maskrcnn.h5")

In [8]:
def grad_cam(model, image_path, layer_name="conv2d_2"):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0) / 255.0

    # Get predictions
    preds = model.predict(img)
    class_idx = np.argmax(preds[0])

    # Get gradients of last conv layer
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        loss = predictions[:, class_idx]
    
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    # Multiply filters by importance
    for i in range(conv_outputs.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    # Create heatmap
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)  # ReLU
    heatmap /= np.max(heatmap)
    
    heatmap = cv2.resize(heatmap, (224, 224))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    original_img = cv2.imread(image_path)
    original_img = cv2.resize(original_img, (224, 224))
    superimposed_img = cv2.addWeighted(original_img, 0.6, heatmap, 0.4, 0)

    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(original_img[:, :, ::-1])
    plt.title("Original Image")
    
    plt.subplot(1, 2, 2)
    plt.imshow(superimposed_img[:, :, ::-1])
    plt.title("Grad-CAM")
    plt.show()

In [9]:
image_path = "dataset/deepfake_no_tumor/generated_deepfake_no_1.png"
grad_cam(model, image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step


AttributeError: The layer sequential has never been called and thus has no defined output.